In [1]:
"""Python WPS execute"""
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
from os import system

In [3]:
wps_url = "http://localhost:8093/wps"
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [4]:
print wps.identification.title

Flyingpigeon


In [5]:
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

subset_continents : 	 Returns only the selected polygon for each input dataset
subset_countries : 	 Returns only the selected polygon for each input dataset
subset_regionseurope : 	 Returns the selected European administrative region defined in the GADM database (v2.5) for each input dataset.
subset_points : 	 Extract timeseries for specified coordinates from gridded datasets
indices_simple : 	 Climate indices based on one single input variable.
indices_percentile : 	 Climate indices based on one single input variable                             and the percentile of a reference period.
weatherregimes_reanalyse : 	 Weather Regimes based on pressure patterns, fetching selected Realayses Datasets
weatherregimes_model : 	 Weather Regimes based on pressure patterns, fetching selected Realayses Datasets
weatherregimes_projection : 	 Weather Regimes detection based on trained reference statistics
analogs_detection : 	 Search for days with analogue pressure pattern for reanalyses data sets
an

# Step1: Fetching occurence data for Fagus Sylvatica

In [6]:
p = wps.describeprocess(identifier='sdm_gbiffetch')
for input in p.dataInputs:
    printInputOutput(input)
    print '\n'

 identifier=taxon_name, title=Tree Species, abstract=Scientific name of tree species, data type=//www.w3.org/TR/xmlschema-2/#string
 Any value allowed
 Default Value: Fagus sylvatica 
 minOccurs=0, maxOccurs=1


 identifier=BBox, title=Bounding Box, abstract=coordinates to define the region for occurence data fetch, data type=BoundingBoxData
 Supported Value: EPSG:4326
 Default Value: EPSG:4326 
 minOccurs=1, maxOccurs=1




In [8]:
execute = wps.execute(
    identifier="sdm_gbiffetch",
    inputs=[
    ('taxon_name','Fagus sylvatica'),
    ('BBox', '-10.0, 30.0, 10.0, 40.0')
    ],
    async=False,
    )


TypeError: execute() got an unexpected keyword argument 'async'

In [7]:
# check process if completed ...
monitorExecution(execute, sleepSecs=5)

for o in execute.processOutputs:
    print o.reference

http://localhost:8090/wpsoutputs/flyingpigeon/output_csv-be40b922-c10d-11e6-86e6-142d277ef1f3.csv
http://localhost:8090/wpsoutputs/flyingpigeon/output_map-be40b922-c10d-11e6-86e6-142d277ef1f3.png


In [8]:
graphic = execute.processOutputs[1].reference
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= graphic)

# Step2: Calculate Indices:

In [14]:
# Define location of input climate data (tas, tasmin, tasmax or pr depending on selected indice)

from os import getenv
HOME = getenv('HOME')

resources = [
    'file://' + HOME + '/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-44/CLMcom/MPI-M-MPI-ESM-LR/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/day/tasmax/v20140520/tasmax_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_day_19960101-20001231.nc',
    'file://' + HOME+'/birdhouse/var/lib/pywps/cache/malleefowl/./cordexesg.dmi.dk/thredds/fileServer/cordex_noncommercial/cordex/output/EUR-44/HMS/CNRM-CERFACS-CNRM-CM5/historical/r1i1p1/HMS-ALADIN52/v1/day/tas/v20140804/tas_EUR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_HMS-ALADIN52_v1_day_20010101-20051231.nc'
    ]

In [15]:
resources

['file:///home/nils/birdhouse/var/lib/pywps/cache/malleefowl/esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-44/CLMcom/MPI-M-MPI-ESM-LR/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/day/tasmax/v20140520/tasmax_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_day_19960101-20001231.nc',
 'file:///home/nils/birdhouse/var/lib/pywps/cache/malleefowl/./cordexesg.dmi.dk/thredds/fileServer/cordex_noncommercial/cordex/output/EUR-44/HMS/CNRM-CERFACS-CNRM-CM5/historical/r1i1p1/HMS-ALADIN52/v1/day/tas/v20140804/tas_EUR-44_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_HMS-ALADIN52_v1_day_20010101-20051231.nc']

In [16]:
p = wps.describeprocess(identifier='sdm_getindices')
for input in p.dataInputs:
    printInputOutput(input)
    print '\n'

 identifier=resources, title=NetCDF File, abstract=NetCDF File, data type=ComplexData
 Supported Value: mimeType=application/x-netcdf, encoding=None, schema=None
 Default Value: mimeType=application/x-netcdf, encoding=None, schema=None 
 minOccurs=1, maxOccurs=500


 identifier=input_indices, title=Indices, abstract=Climate indices related to growth conditions                 of tree species, data type=//www.w3.org/TR/xmlschema-2/#string
 Allowed Value: CDD_AMJJAS
 Allowed Value: CFD_ONDJFM
 Allowed Value: FD_ONDJFM
 Allowed Value: FD_April
 Allowed Value: FD_May
 Allowed Value: FD_June
 Allowed Value: GD4_yr
 Allowed Value: ID_yr
 Allowed Value: PRCPTOT_yr
 Allowed Value: PRCPTOT_ONDJFM
 Allowed Value: PRCPTOT_AMJJAS
 Allowed Value: PRCPTOT_JJA
 Allowed Value: PRCPTOT_MAM
 Allowed Value: PRCPTOT_JJA
 Allowed Value: RR1_yr
 Allowed Value: SU_yr
 Allowed Value: TG_yr
 Allowed Value: TG_AMJJAS
 Allowed Value: TG_ONDJFM
 Allowed Value: TG_JJA
 Allowed Value: TNn_yr
 Allowed Value: TNn_AMJ

In [17]:
execute = wps.execute(
    identifier="sdm_getindices",
    inputs=[
    ('resources',resources[0]),
    ('resources',resources[1]),
    ('input_indices', 'TG_JJA'),
    ('input_indices', 'SU_yr'),
    ('input_indices', 'ID_yr')
    ],
    #outputs = [('logout', 'True'),('tarout', 'True') ,('anomaliesout', 'True') ]
    )
# check process if completed ...
monitorExecution(execute, sleepSecs=5)
for o in execute.processOutputs:
    print o.reference

http://localhost:8090/wpsoutputs/flyingpigeon/output_indices-94fb1c2e-c09f-11e6-b657-142d277ef1f3.tar
